## Non Compliance Experiment=1
Test top norms for different w_nc

In [1]:
from IPython.utils import io
from tqdm.notebook import tqdm
from pathlib import Path

In [2]:
def write_log(output,path):
    with open(Path(path), 'w') as f:
        f.write(output)

In [6]:
%%time
with io.capture_output() as exp_nc_0_log:
    !python ../scripts/nc_experiments.py -exp exp_nc0 -w_nc 0.0 -n_threads 6
write_log(exp_nc_0_log.stdout,"../data_nc/exp_nc0/exp_nc_0_run.log")  

CPU times: user 7.88 s, sys: 2.16 s, total: 10 s
Wall time: 28min 3s


In [7]:
!grep "Time for" ../data_nc/exp_nc0/exp_nc_0_run.log

In [97]:
!dvc add ../data_nc/exp_nc0/
!git add --all
!git commit -m "Results gen for dummy_exp_nc_0"

Adding...                                                                       
!
  0%|          |Computing file/dir hashes (only done 0/11 [00:00<?,      ?md5/s]
                                                                                
!
Saving exp_nc0                                        |0.00 [00:00,     ?file/s]
100% Add|██████████████████████████████████████████████|1/1 [00:01,  1.24s/file]

To track the changes with git, run:

	git add ../data_nc/.gitignore ../data_nc/exp_nc0.dvc
  0% Uploading|                                     |0/12 [00:00<?,     ?file/s]
!
  0%|          |../data_nc/exp_nc0/chain_poster0.00/5.03M [00:00<?,       ?it/s]

!

  0%|          |../data_nc/exp_nc0/exp_nc_0_run0.00/2.01M [00:00<?,       ?it/s]


!


  0%|          |../data_nc/exp_nc0/precision_re0.00/9.73k [00:00<?,       ?it/s]


 84%|████████▍ |../data_nc/exp_nc0/precisi8.00k/9.50k [00:00<00:00,    11.8kB/s]
  0%|          |../data_nc/exp_nc0/chain_p8.00k/4.80M [00:00<08:05,    10.3kB/s]

In [98]:
!git add --all
!git commit -m "Results gen for dummy_exp_nc_0"

[non_compliance 0660bd2] Results gen for dummy_exp_nc_0
 3 files changed, 43 insertions(+), 33 deletions(-)
 create mode 100644 data_nc/.gitignore
 create mode 100644 data_nc/exp_nc0.dvc


In [100]:
w_nc_list=[0.0,0.05,0.1,0.2,0.3,0.4,0.5]

In [101]:
for w_nc in tqdm(w_nc_list,"Loop Over diff w_nc"):
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    with io.capture_output() as exp_log:
        !python ../scripts/nc_experiments.py -exp $exp_dir_name -w_nc $w_nc  -n_threads 6
    write_log(exp_log.stdout,"../data_nc/{}/run.log".format(exp_dir_name)) 

In [102]:
w_nc

0.5